In [2]:
import cv2
import os
import pickle
import mediapipe as mp
from tqdm import tqdm
from IPython.display import clear_output

2025-03-24 19:40:00.427484: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742812800.443027    6664 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742812800.448861    6664 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742812800.460774    6664 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742812800.460793    6664 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1742812800.460794    6664 computation_placer.cc:177] computation placer alr

In [3]:
# 데이터셋 경로
DATASET_DIR = '/home/shin/deeplearning-repo-1/dataset/stand_knee_raise'

# 좌표 추출 운동 설정
exericse_name = "stand_knee_raise"

pickle_name = exericse_name + ".pkl"

exericse_type = ["squat", "pushup", "stand_knee_raise", "shoulder_press"]
exericse_label = exericse_type.index(exericse_name)

# 시퀀스 사이즈
SEQUENCE_LENGTH = 20

# mediapipe 선언
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

sequences = []
labels = []
landmark_points = [ 11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28 ]

with mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5) as pose:
    clear_output(wait=True)
    for video_file in tqdm(os.listdir(DATASET_DIR)):
        video_path = os.path.join(DATASET_DIR, video_file)
        cap = cv2.VideoCapture(video_path)
        frames = []
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = pose.process(frame_rgb)
            if results.pose_landmarks:
                landmarks = []

                lm = results.pose_landmarks.landmark

                # 기준점
                neck_x = (lm[11].x + lm[12].x) / 2
                neck_y = (lm[11].y + lm[12].y) / 2
                for idx in landmark_points:
                    x = lm[idx].x - neck_x
                    y = lm[idx].y - neck_y
                    landmarks.extend([x, y])
                frames.append(landmarks)
                if len(frames) >= SEQUENCE_LENGTH:
                    sequences.append(frames[-SEQUENCE_LENGTH:])
                    labels.append(exericse_label)
        cap.release()

  0%|          | 0/10 [00:00<?, ?it/s]W0000 00:00:1742812803.473510    6817 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742812803.482334    6794 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742812803.530604    6810 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742812803.536120    6795 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1742812803.577063    6811 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.
100%|██████████| 10/10 [01:

In [4]:
with open(pickle_name, 'wb') as f:
    pickle.dump((sequences, labels), f)

    print(exericse_name + " 데이터 저장 완료!")

stand_knee_raise 데이터 저장 완료!
